<a href="https://colab.research.google.com/github/janoPig/HROCH/blob/main/examples/Symbolic_Regression_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Symbolic Regression Demo


1.   Setup
2.   Basic example ground-truth problem
3.   Basic example blackbox problem
4.   Use feature importances from bbox model
5.   Custom instructions set
6.   Simple binary clasification with lt/gt
7.   Fuzzy regression
8.   Classification with fuzzy logic - parity dataset



## Setup

In [200]:
%pip install -U HROCH
#Penn Machine Learning Benchmarks
%pip install -U git+https://github.com/EpistasisLab/pmlb



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/EpistasisLab/pmlb to /tmp/pip-req-build-tpg1i5_4
  Running command git clone --filter=blob:none --quiet https://github.com/EpistasisLab/pmlb /tmp/pip-req-build-tpg1i5_4
  Resolved https://github.com/EpistasisLab/pmlb to commit 8df469eb67d139d3f2ce4418e6fb7cf10ccbf84e
  Preparing metadata (setup.py) ... done


In [201]:
import pandas as pd
import numpy as np
import sympy as sp
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from pmlb import fetch_data
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from HROCH import PHCRegressor

## Basic example ground-truth problem

feynman_III_7_38 dataset from pmlb

Formula: omega = 2 * mom * B/(h/(2 * pi))


In [202]:
def get_eq(X : pd.DataFrame, expr : str):
    model_str = str(sp.parse_expr(expr))
    mapping = {'x'+str(i+1): k for i, k in enumerate(X.columns)}
    new_model = model_str
    for k, v in reversed(mapping.items()):
        new_model = new_model.replace(k, v)

    return new_model

dataset = fetch_data('feynman_III_7_38')
Y = np.ravel(pd.DataFrame(dataset, columns=['target']).to_numpy())
X = dataset.drop(columns=['target'])
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), Y, train_size=0.75, test_size=0.25, random_state=42)

In [203]:
reg = PHCRegressor(num_threads=1, time_limit = 0.0, iter_limit=1000000, random_state=42)
reg.fit(X_train, y_train)

yp_train = reg.predict(X_train)
r2_train = r2_score(y_train, yp_train)
rms_train = np.sqrt(mean_squared_error(y_train, yp_train))

yp = reg.predict(X_test)
r2 = r2_score(y_test, yp)
rms = np.sqrt(mean_squared_error(y_test, yp))

print(f'train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')
print(f'eq: {get_eq(X, reg.sexpr)}')

train: r2=0.9999999999999873 rms=4.039649995870836e-06 test: r2=0.9999999999999877 rms=4.028888405992124e-06
eq: 12.5663709640502929688*mom*B/h


## Basic example blackbox problem

588_fri_c4_1000_100 dataset from pmlb

In [204]:
dataset = fetch_data('588_fri_c4_1000_100')
Y = np.ravel(pd.DataFrame(dataset, columns=['target']).to_numpy())
X = dataset.drop(columns=['target'])
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), Y, train_size=0.75, test_size=0.25, random_state=42)

In [205]:
reg = PHCRegressor(num_threads=1, time_limit = 0.0, iter_limit=1000000, random_state=42)
reg.fit(X_train, y_train)

yp_train = reg.predict(X_train)
r2_train = r2_score(y_train, yp_train)
rms_train = np.sqrt(mean_squared_error(y_train, yp_train))

yp = reg.predict(X_test)
r2 = r2_score(y_test, yp)
rms = np.sqrt(mean_squared_error(y_test, yp))

print(f'PHCRegressor train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')
print(f'eq: {get_eq(X, reg.sexpr)}')

PHCRegressor train: r2=0.6705863184329303 rms=0.5724199024786217 test: r2=0.5941229429930125 rms=0.6408433039138661
eq: 0.58182519674301147461*(oz2*(oz1 + oz5) - 2.91932797431945800781)*sin(oz2)


## Use feature importances from bbox model

For example, we can use the feature importances from RandomForestRegressor to try to speed up the search process. During mutation, the PHCRegressor will select the most important features with higher probability. 

In [206]:
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

yp_train = reg.predict(X_train)
r2_train = r2_score(y_train, yp_train)
rms_train = np.sqrt(mean_squared_error(y_train, yp_train))

yp = rf.predict(X_test)
r2 = r2_score(y_test, yp)
rms = np.sqrt(mean_squared_error(y_test, yp))

print(f'RandomForestRegressor train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')

RandomForestRegressor train: r2=0.6705863184329303 rms=0.5724199024786217 test: r2=0.8303892163473732 rms=0.4142679448325175


In [207]:
probs = np.power(rf.feature_importances_, 2.0)
reg = PHCRegressor(num_threads=1, time_limit = 0.0, iter_limit=1000000, random_state=42, feature_probs=probs)

reg.fit(X_train, y_train)
yp_train = reg.predict(X_train)
r2_train = r2_score(y_train, yp_train)
rms_train = np.sqrt(mean_squared_error(y_train, yp_train))

yp = reg.predict(X_test)
r2 = r2_score(y_test, yp)
rms = np.sqrt(mean_squared_error(y_test, yp))

print(f'PHCRegressor train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')
print(f'eq: {get_eq(X, reg.sexpr)}')

PHCRegressor train: r2=0.743787720763619 rms=0.504828516226028 test: r2=0.7067059821487739 rms=0.5447612427637081
eq: -0.81791335344314575195*oz2 + 0.81791335344314575195*oz5 - 0.81791335344314575195*sin(oz1 + 1.9202204843009088797*oz2 - 0.52077353000640869141)


## Custom instructions set

Limit the search to specific mathematical operations. Each math instruction has a defined probability used by the mutation operator.

|**Supported instructions**||
| ----------- | ----------- |
|**math**|add, sub, mul, div, inv, minv, sq2, pow, exp, log, sqrt, cbrt, aq|
|**goniometric**|sin, cos, tan, asin, acos, atan, sinh, cosh, tanh|
|**other**|nop, max, min, abs, floor, ceil, lt, gt, lte, gte|
|**fuzzy**|f_and, f_or, f_xor, f_impl, f_not, f_nand, f_nor, f_nxor, f_nimpl|

In [208]:
instr_set={'add': 1.0, 'mul': 1.0, 'div':0.01, 'sin':0.1}
reg = PHCRegressor(num_threads=1, time_limit = 0.0, iter_limit=1000000, random_state=42, feature_probs=probs, problem=instr_set)

reg.fit(X_train, y_train)
yp_train = reg.predict(X_train)
r2_train = r2_score(y_train, yp_train)
rms_train = np.sqrt(mean_squared_error(y_train, yp_train))

yp = reg.predict(X_test)
r2 = r2_score(y_test, yp)
rms = np.sqrt(mean_squared_error(y_test, yp))

print(f'train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')
print(f'eq: {get_eq(X, reg.sexpr)}')

train: r2=0.8826225841667946 rms=0.34169307794085274 test: r2=0.8569455124038241 rms=0.38045679277078087
eq: 0.43304610252380371094*oz4 + (0.5332279205322265625*oz2 + 1.15445457330451972666)*sin(oz1*(0.5332279205322265625*oz2 + 1.15445457330451972666) + 1.17703820286578775267*oz2 + 2.54832330403152253522)


## Simple binary clasification with lt/gt

In [209]:
X = np.random.normal(loc=0.0, scale=10.0, size=(4000, 100))
y = (0.5*X[:, 0]**2 >= 1.5*X[:, 1])*1.0

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_predicted = dtc.predict(X_test)
test_mse = mean_squared_error(y_predicted, y_test)
test_r2 = r2_score(y_predicted, y_test)
print(f'DecisionTreeClassifier: mse= {test_mse} r2= {test_r2}')

probs = np.power(dtc.feature_importances_, 2.0)
instr_set={'add': 1.0, 'sub': 1.0, 'mul': 1.0, 'lt':0.1, 'gt':0.1, 'lte':0.1, 'gte':0.1}
reg = PHCRegressor(num_threads=1, time_limit = 0.0, iter_limit=10000000, random_state=42,feature_probs=probs, problem=instr_set)
reg.fit(X_train, y_train)

# predict
y_predicted = reg.predict(X_test)
y_predicted = (y_predicted > 0.5)*1.0
test_mse = mean_squared_error(y_predicted, y_test)
test_r2 = r2_score(y_predicted, y_test)

print(f'PHCRegressor: mse= {test_mse} r2= {test_r2} eq= {str(reg.sexpr)} ')

DecisionTreeClassifier: mse= 0.023 r2= 0.8428059815741272
PHCRegressor: mse= 0.0 r2= 1.0 eq= ((((x1*x1)+x2)+(((x2-(((0.00058132671983912587)-x1)*((0.00058132671983912587)-x1)))-(((0.00058132671983912587)-x1)*((0.00058132671983912587)-x1)))+x2))<(0.00797392893582582474))
 


## Fuzzy regression

Let's create a data set with 40 elements and satisfying the equation
y = ((X1 & X16) | (!X4 & X19)) & (X23 | X26)

In [210]:
X = np.random.uniform(low=0.0, high=1.0, size=(4000, 40))
A = X[:, 0] * X[:, 15]
B = (1.0 - X[:, 3]) * X[:, 18]
C = A + B - A * B  # A or b
D = X[:, 22] + X[:, 25] - X[:, 22] * X[:, 25]
y = C * D

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

Try run PHCRegressor find correct equation.

In [211]:
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)

y_predicted = reg.predict(X_test)
test_mse = mean_squared_error(y_predicted, y_test)
test_r2 = r2_score(y_predicted, y_test)

print(f'DecisionTreeRegressor: mse= {test_mse} r2= {test_r2}')

probs = np.power(reg.feature_importances_, 2.0)

reg = PHCRegressor(num_threads=1, time_limit=0.0, iter_limit=20000000, random_state=42, problem='fuzzy')
reg.fit(X_train, y_train)

# predict
y_predicted = reg.predict(X_test)
test_mse = mean_squared_error(y_predicted, y_test)
test_r2 = r2_score(y_predicted, y_test)

print(f'PHCRegressor: mse= {test_mse} r2= {test_r2} eq= {str(reg.sexpr)} ')

DecisionTreeRegressor: mse= 0.009374327113953654 r2= 0.7790637321802895
PHCRegressor: mse= 9.480329266950621e-16 r2= 0.999999999999978 eq= ((x23|(x26|((x20&((x16&(-0.00010815345012815669))&(x16&(-0.00010815345012815669))))|(x20&((x16&(-0.00010815345012815669))&(x16&(-0.00010815345012815669)))))))&(((!x4)&(((x16&(-0.00010815345012815669))&(x16&(-0.00010815345012815669)))|x19))|((x1&x16)^((x16&(-0.00010815345012815669))&(x16&(-0.00010815345012815669))))))
 


## Classification with fuzzy logic - parity dataset

A good simple example is the parity5 and parity5+5 dataset from pmlb. 
PHCRegressor will find equations (((x5^(x4^x3))^x1)^x2) or similar that can be simplified to this form. The equation Xor fits the parity calculation perfectly. The DecisionTreeClassifier and RandomForestClassifier fit the training data with an r2 score of 1.0, but absolutely not the test data.

Because the parity5 dataset is very small we repeat the experiment 10 times

In [212]:
datasets = [(fetch_data('parity5'), 'parity5'), (fetch_data('parity5+5'), 'parity5+5')]
random_states = [42, 1083, 20133, 35879, 45688, 211565, 1212248, 58985945, 48994485, 5454544]
classifiers = {PHCRegressor: {'problem':'fuzzy', 'iter_limit':5000000, 'num_threads':1}, DecisionTreeClassifier: {}, RandomForestClassifier: {}}

for classifier, params in classifiers.items():
  print(classifier.__name__)
  print('='*20)
  for dataset, dataset_name in datasets:
    print(dataset_name)
    print('-'*20)
    Y = np.ravel(pd.DataFrame(dataset, columns=['target']).to_numpy())
    X = dataset.drop(columns=['target']).to_numpy()
    for rs in random_states:
      X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, test_size=0.25, random_state=rs)
      clf = classifier(random_state=rs, **params)
      clf.fit(X_train, y_train)
      yp_train = clf.predict(X_train)
      if classifier is PHCRegressor:
        yp_train = (yp_train > 0.5)*1.0
      r2_train = r2_score(y_train, yp_train)
      rms_train = np.sqrt(mean_squared_error(y_train, yp_train))
      yp = clf.predict(X_test)
      if classifier is PHCRegressor:
        yp = (yp > 0.5)*1.0
      r2 = r2_score(y_test, yp)
      rms = np.sqrt(mean_squared_error(y_test, yp))
      print(f'train: r2={r2_train} rms={rms_train} test: r2={r2} rms={rms}')
      if classifier is PHCRegressor:
        print(f'eq: {clf.sexpr}')

PHCRegressor
parity5
--------------------
train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: ((0.00000000000000000000)|(x2^((x5|x5)^(x4^(x1^x3)))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: ((x4^x2)^(x5^((0.00000000000000000000)|((x1&x1)^x3))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (((x2&x2)^(((x1&x1)^x3)^(x5^x4)))&((x2&x2)^(((x1&x1)^x3)^(x5^x4))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (x4^(x5^(x3^(x1^x2))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (!((x5^(x4&x4))^((x2^(!x3))^x1)))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (((x5^(x4^x3))^x1)^x2)

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (x2^(((x3^x1)^(x4^x5))|((x3^x1)^(x4^x5))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (x5^((x4&(!(!x4)))^(x3^(x2^x1))))

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (((x3^(x4^x2))^((0.00000000000000000000)|x1))^x5)

train: r2=1.0 rms=0.0 test: r2=1.0 rms=0.0
eq: (((0.00000000000000000000)|x1)^(x4^((x3^x2)^x5)))

parity5+5
--------------------
train: r2=1.